In [ ]:
!pip3 install --user kafka-python

In [ ]:
import json
with open('properties.json', 'r') as f:
    properties = json.load(f)
properties

In [ ]:
brokers = '{}-cp-kafka:9092'.format(properties['pnda-helm-release'])
topic = properties['topic']

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from kafka.errors import KafkaError
import logging
import json
import datetime
import time
import random

ids = ['1234','4321','2314','4123']

producer = KafkaProducer(bootstrap_servers=[brokers])
print("Starting Streaming Openmetrics to Kafka. Interrupt the Kernel to Stop.")
while True:
  for id in ids:
    openmetric = 'bitrate{{id="{id}"}} {value} {timestamp}'.format(id=random.choice(ids),value=int(random.random()*1E7),timestamp=int(datetime.datetime.now().timestamp()*1000)).encode('utf-8')
    future = producer.send('openmetrics', openmetric)
    openmetric = 'signal_strength{{id="{id}"}} {value:.2f} {timestamp}'.format(id=random.choice(ids),value=random.random()*10,timestamp=int(datetime.datetime.now().timestamp()*1000)).encode('utf-8')
    future = producer.send('openmetrics', openmetric)
  try:
    record_metadata = future.get(timeout=10)
  except KafkaError:
    # Decide what to do if produce request failed...
    logging.exception('Kafka Exception')
    pass
  time.sleep(5)